In [5]:
import sys
sys.path.insert(0, '../')
import importlib
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
import cobra
from coralme.util.helpers import get_nlp,optimize
from contextlib import contextmanager
import os

@contextmanager
def suppress_output():
    # Save file descriptors
    stdout_fd = os.dup(1)
    stderr_fd = os.dup(2)
    
    # Open null device
    null_fd = os.open(os.devnull, os.O_RDWR)
    
    # Redirect stdout and stderr to null device
    os.dup2(null_fd, 1)
    os.dup2(null_fd, 2)
    
    try:
        yield
    finally:
        # Restore file descriptors
        os.dup2(stdout_fd, 1)
        os.dup2(stderr_fd, 2)
        
        # Clean up
        os.close(null_fd)
        os.close(stdout_fd)
        os.close(stderr_fd)
            
# load the model and create the nlp and dictionaries for solvers
me = coralme.io.pickle.load_pickle_me_model('../species_files/Zymomonas_mobilis/outputs/MEModel-step3-zymomonas-TS.pkl')
nlp = get_nlp(me)
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}
default_ups = {
    exch : nlp.xu[rxn_index_dct[exch]] for exch in rxn_index_dct.keys()
}
default_lows = {
    exch : nlp.xl[rxn_index_dct[exch]] for exch in rxn_index_dct.keys()
}

In [6]:
# loop through all exchanges, shut off one at a time to figure out what can be cut
carbon_exchanges = [
    'EX_gcald_e', 'EX_glc_e', 'EX_glcn_e', 'EX_glyc_e', 
    'EX_succ_e', 'EX_sucr_e', 'EX_fum_e', 'EX_fru_e', 
    'EX_g3pc_e', 'EX_g3pe_e', 'EX_g3pg_e', 'EX_g3pi_e', 
    'EX_g3ps_e', 'EX_gal_e', 'EX_gal__bD_e', 'EX_gdp_e', 
    'EX_gln__L_e', 'EX_glu__L_e', 'EX_glyc3p_e', 'EX_lac__D_e', 
    'EX_xyl__D_e', 'EX_glycogen'
]
all_exchanges = [val for val in rxn_index_dct.keys() if 'EX_' in val]


exchanges_use = all_exchanges
i = 0
for run in exchanges_use:
    # turn everything to defaults
    for exch in exchanges_use:
        nlp.xl[rxn_index_dct[exch]] = default_lows[exch]#lambda x : -10
        nlp.xu[rxn_index_dct[exch]] = default_ups[exch]#lambda x : 10

    # turn off run
    nlp.xl[rxn_index_dct[run]] = lambda x : 0
    nlp.xu[rxn_index_dct[run]] = lambda x : 0
    
    # now run the model
    with suppress_output():
        sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.1, min_mu = 0.01,
    		tolerance = 1e-4, maxIter = 1, precision = 'double', verbose = False, basis = None)
    i += 1
    print(str(i)+' / '+str(len(exchanges_use)), end = ' : ')
    if str(type(sol)) != '<class \'NoneType\'>':
        # growth occured!
        print(run+' can grow without = ', end = '')
        print(sol.objective_value)
    else:
        print(run+' cannot grow without')

1 / 93 : EX_3dhq_e can grow without = 0.055
2 / 93 : EX_3dhsk_e can grow without = 0.055
3 / 93 : EX_4hbz_e can grow without = 0.055
4 / 93 : EX_ac_e cannot grow without
5 / 93 : EX_acald_e can grow without = 0.055
6 / 93 : EX_actn__S_e can grow without = 0.055
7 / 93 : EX_ade_e can grow without = 0.055
8 / 93 : EX_anhgm_e can grow without = 0.055
9 / 93 : EX_arg__L_e can grow without = 0.055
10 / 93 : EX_asn__L_e can grow without = 0.055
11 / 93 : EX_asp__L_e can grow without = 0.055
12 / 93 : EX_btn_e can grow without = 0.055
13 / 93 : EX_butso3_e can grow without = 0.055
14 / 93 : EX_ca2_e can grow without = 0.055
15 / 93 : EX_cd2_e can grow without = 0.055
16 / 93 : EX_cgly_e can grow without = 0.055
17 / 93 : EX_chol_e can grow without = 0.055
18 / 93 : EX_cl_e can grow without = 0.055
19 / 93 : EX_co2_e cannot grow without
20 / 93 : EX_cobalt2_e can grow without = 0.055
21 / 93 : EX_cu_e can grow without = 0.055
22 / 93 : EX_cynt_e can grow without = 0.055
23 / 93 : EX_etha_e can

In [7]:
# check for limiters
for rxn, flux in sol.fluxes.items():
    #if 'EX_' not in rxn: continue
    if flux == 0: continue
    if flux == nlp.xl[rxn_index_dct[rxn]](0):
        print(rxn, end = ' = ')
        print(flux)
    if flux == nlp.xu[rxn_index_dct[rxn]](0):
        print(rxn, end = ' = ')
        print(flux)

EX_glc_e = -70.0
EX_glc_e = -70.0
ATPM = 3.7138263401347
TS_cobalt2_c = -10.0
